In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.pipeline import Pipeline as imb_pipeline
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTENC, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.dummy import DummyClassifier


data = pd.read_csv("healthcare-dataset-stroke-data.csv")

balanced_data = data.dropna()
balanced_data = balanced_data.drop(columns= "id")
balanced_data = balanced_data[balanced_data.age > 20]
balanced_data = balanced_data.drop(balanced_data.index[balanced_data["gender"] == "Other"])
balanced_data = balanced_data.drop(balanced_data.index[balanced_data["work_type"] == "Never_worked"])

features = ['gender', 'age', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'avg_glucose_level', "bmi", "smoking_status"]
cat_columns = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status', 'hypertension', 'heart_disease']
num_columns = ['age', 'avg_glucose_level', 'bmi']

bal_data_train, bal_data_test = train_test_split(balanced_data, train_size = 0.8, random_state=1, stratify= balanced_data["stroke"])
bal_data_train, bal_data_val = train_test_split(bal_data_train, train_size = 0.8, random_state=1)

X_train = bal_data_train.drop('stroke', axis = 1)
y_train = bal_data_train['stroke']

X_val = bal_data_val.drop('stroke', axis = 1)
y_val = bal_data_val['stroke']

X_test = bal_data_test.drop('stroke', axis = 1)
y_test = bal_data_test['stroke']

In [ ]:
preprocessing = ColumnTransformer([
    ('one-hot-encoder', OneHotEncoder(sparse_output=False), cat_columns),
    ('scaler', StandardScaler(), num_columns)
])

In [ ]:
gs_pipeline = imb_pipeline(steps= [
        ("smotenc", SMOTENC(cat_columns, random_state = 1)),
        ('preprocessing', preprocessing),
        ('classifier', DummyClassifier(random_state = 1, strategy="stratified")) #Baseline comparator
       ])


param_grid = [
                {
                    "smotenc__sampling_strategy": [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
                    "classifier": [SVC(random_state=1)],
                    'classifier__C': [1, 10, 100, 1000], 
                    'classifier__kernel': ['linear', "rbf"],
                    'classifier__gamma': [0.001, 0.0001]
                },

                {
                    
                }


]


gs = GridSearchCV(
    gs_pipeline,
    param_grid,
    cv = 3,
    scoring= "f1",
    n_jobs= -1
)

gs.fit(X_train, y_train)

In [ ]:
print("Best params:", gs.best_params_)
print("Score:", gs.best_score_)

Saved best estimator so I dont have to run it for 15 minutes again

In [ ]:
import joblib

best2 = gs.best_estimator_
joblib.dump(best2, "best_estimator2.pkl")

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

print("Training set")
print("F1", f1_score(y_train, best2.predict(X_train)))
print("Recall",recall_score(y_train, best2.predict(X_train)))
print("Precision",precision_score(y_train, best2.predict(X_train)))
print(confusion_matrix(y_train, best2.predict(X_train)))

print()
print("Val set")
print("F1", f1_score(y_val, best2.predict(X_val)))
print("Recall",recall_score(y_val, best2.predict(X_val)))
print("Precision",precision_score(y_val, best2.predict(X_val)))
print(confusion_matrix(y_val, best2.predict(X_val)))

print()
print("Test set")
print("F1", f1_score(y_test, best2.predict(X_test)))
print("Recall", recall_score(y_test, best2.predict(X_test)))
print("Precision",precision_score(y_test, best2.predict(X_test)))
print(confusion_matrix(y_test, best2.predict(X_test)))